In [16]:
#loading dependencies
import numpy as np
import math
from pprint import pprint
import random
import operator
%matplotlib inline
import matplotlib as mpl

import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import matplotlib #only needed to determine Matplotlib version number

from os import listdir
from os.path import isfile, join
import csv

In [35]:
def load_data(path):   
    # ==> Loading Merged Station Data
    data = pd.read_json(path)
    # ==> Rebuilding Multiindex
    data.set_index(["number",'last_update'],inplace=True,drop=True)
    # ==> Sort index by number & timestamp
    data.sort_index(inplace=True)
    return data

In [36]:
data = load_data('challenge_data/stations_merged.json')

In [31]:
data.index.values[0:100]

array([(1010, 1441065103000), (1010, 1441065707000), (1010, 1441066310000),
       (1010, 1441066913000), (1010, 1441067517000), (1010, 1441068120000),
       (1010, 1441068724000), (1010, 1441069327000), (1010, 1441069931000),
       (1010, 1441070535000), (1010, 1441071138000), (1010, 1441071742000),
       (1010, 1441072348000), (1010, 1441072952000), (1010, 1441073555000),
       (1010, 1441074158000), (1010, 1441074762000), (1010, 1441075365000),
       (1010, 1441075969000), (1010, 1441076572000), (1010, 1441077176000),
       (1010, 1441077780000), (1010, 1441078383000), (1010, 1441078986000),
       (1010, 1441079590000), (1010, 1441080194000), (1010, 1441080797000),
       (1010, 1441081401000), (1010, 1441082004000), (1010, 1441082608000),
       (1010, 1441083211000), (1010, 1441083815000), (1010, 1441084418000),
       (1010, 1441085024000), (1010, 1441085424000), (1010, 1441086028000),
       (1010, 1441086631000), (1010, 1441087736000), (1010, 1441088239000),
       (1010